<a href="https://colab.research.google.com/github/coldsober-irene/ASSIGNMENTS/blob/main/dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import block_diag


In [50]:
%%capture
!pip install control


In [48]:
# System parameters and simulation parameters
A = np.array([[0.9638, 0.0162], [0.0162, 0.9627]])  # System matrix
B = np.array([[0.0031, 0], [0, 0.0031]])  # Input matrix
C = np.eye(2)
Np = 50  # Prediction horizon
Nc = 10

#discretization

In [66]:
import numpy as np
import control

def c2d(Ac = np.array([[-0.370, 0.168], [0.168, -0.38]]),
    Bc = np.array([[0.03125, 0], [0, 0.03125]]),
    Cc = np.array([[1, 0], [0, 1]]),
    Dc = np.array([[0, 0], [0, 0]]),
    Ts = 0.1):

    # Create a state-space system
    sys_continuous = control.ss(Ac, Bc, Cc, Dc)
    # Discretize the system
    sys_discrete = sys_continuous.sample(Ts, method='zoh')

    # The discretized system matrices can be accessed as follows:
    Ad = sys_discrete.A
    Bd = sys_discrete.B
    Cd = sys_discrete.C
    Dd = sys_discrete.D
    return Ad, Bd, Cd, Dd

def Augment(Ad, Bd, Cd):
  # Get the sizes
  row1, col1 = Cd.shape
  d, cb = Bd.shape

  # Create the augmented matrices
  A = np.eye(col1 + row1)
  A[:col1, :d] = Ad
  A[:col1, col1:col1 + row1] = Bd

  B = np.zeros((col1 + row1, cb))
  B[:col1, :] = Bd
  B[col1:col1 + d, :] = np.eye(d)

  C = np.zeros((row1, col1 + row1))
  C[:, :col1] = Cd
  return A, B, C

$Y = Fx(k_i) + \Phi \Delta U $

computing $F$ and $\Phi$

In [67]:
Ad, Bd, Cd, _ = c2d()
A, B, C = Augment(Ad, Bd, Cd)
Np = 10

In [77]:
def ComputeF_Phi(A, B, C, Np):

  """A,B,C are the auPhimented matrices
     Np prediction horizon Np > 8
  """
  # Compute the sizes
  F_r, F_c = np.shape(np.dot(C, A))

  # Initialize F
  F = np.zeros((Np*F_r, F_c))

  # First row of F
  F[:F_r, :] = np.dot(C, A)

  # Compute the remaininPhi rows of F
  for k in range(F_r, Np*F_r, F_r):
      F[k:k+F_r, :] = np.dot(F[k-F_r:k, :], A)

  # Compute the sizes
  Phi_r, Phi_c = np.shape(np.dot(C, B))

  # Initialize Phi
  Phi = np.zeros((Np*Phi_r, Nc*Phi_c))

  # First row of Phi
  Phi[:Phi_r, :Phi_c] = np.dot(C, B)

  # Compute the remaininPhi rows of Phi
  for k in range(Phi_r, Np*Phi_r, Phi_r):
      p = (k - Phi_r) // Phi_r + 1
      Phi[k:k+Phi_r, :Phi_c] = np.dot(C, np.linalg.matrix_power(A, p).dot(B))

  # Shift the columns of Phi
  for k in range(Phi_c, Nc*Phi_c, Phi_c):
      shift = Phi_r * (k - Phi_c) // Phi_c + Phi_r
      Phi[:, k:k+Phi_c] = np.roll(Phi[:, :Phi_c], shift)
      Phi[:shift, k:k+Phi_c] = np.zeros((shift, Phi_c))
  return F, Phi

In [ ]:
F, Phi = ComputeF_Phi(A, B, C, Np = 10)
Phi